There appears to be something different about the HTML format of some patents (mostly EPO and US patents), which I should try to figure out

In [7]:
import pandas as pd
import requests
from lxml import html
from random import randint
import time
from time import sleep
import json

In [ ]:
# should somehow not scrape the parts that entail mostly genetic sequences

## Getting the ids

In [9]:
# opening the patents that mention 'accession'
df_keys = pd.read_csv(R'X:\5_Research\Paul\data\genes_of_technology\full_patent_data_lens_marine_2689_familychecked.csv', index_col=0)

In [10]:
df_keys
# the patent name section is flawed, some zeros are missing at times

,patent_id,doc_key,lens_id_new,date_published,families,codes,abstract,claims,legal_status,publication_type,lens_id_old,organism,patent_url_old,patent_url_new
0,TWI352737B,TW_I352737_B_20111121,000-950-825-379-991,2011-11-21,{'simple_family': {'members': [{'document_id':...,"['C07H21/04', 'A01K67/027', 'C12N9/16', 'C12N9...",[{'text': 'The current invention reports a pro...,NaN,"{'granted': True, 'grant_date': '2011-11-21', ...",GRANTED_PATENT,154-387-245-471-34X,Aequorea victoria,https://patents.google.com/patent/JP2010531647...,https://patents.google.com/patent/TWI352737B/en
1,US9410134B2,US_9410134_B2_20160809,000-959-609-762-989,2016-08-09,{'simple_family': {'members': [{'document_id':...,"['C12N15/113', 'A01H3/04', 'A01N57/16', 'A01N5...",[{'text': 'The present invention relates to a ...,[{'claims': [{'claim_text': ['1. A nucleic aci...,"{'granted': True, 'grant_date': '2016-08-09', ...",GRANTED_PATENT,000-959-609-762-989,Odobenus rosmarus divergens,https://patents.google.com/patent/US9410134B2/...,https://patents.google.com/patent/US9410134B2/...
2,US9422558B2,US_9422558_B2_20160823,003-170-265-500-408,2016-08-23,{'simple_family': {'members': [{'document_id':...,"['C07H21/04', 'C12Q1/68']",[{'text': 'The present invention provides nove...,"[{'claims': [{'claim_text': [""1. A method of p...","{'granted': True, 'grant_date': '2016-08-23', ...",GRANTED_PATENT,003-170-265-500-408,Rhizophora mucronata,https://patents.google.com/patent/US9422558B2/...,https://patents.google.com/patent/US9422558B2/...
3,US8420312B2,US_8420312_B2_20130416,004-053-513-540-817,2013-04-16,{'simple_family': {'members': [{'document_id':...,"['C12N15/29', 'C12N5/04', 'C12N15/82', 'C12N15...",[{'text': 'Described herein is a combination c...,[{'claims': [{'claim_text': ['1. A method to d...,"{'granted': True, 'grant_date': '2013-04-16', ...",GRANTED_PATENT,004-053-513-540-817,Trichechus manatus latirostris,https://patents.google.com/patent/US8420312B2/...,https://patents.google.com/patent/US8420312B2/...
4,US7026526B2,US_7026526_B2_20060411,004-902-052-988-996,2006-04-11,{'simple_family': {'members': [{'document_id':...,"['C12Q1/68', 'G01N33/53']","[{'text': ""Methods and constructs for the intr...",[{'claims': [{'claim_text': ['1. A DNA constru...,"{'granted': True, 'grant_date': '2006-04-11', ...",GRANTED_PATENT,045-496-187-872-188,Pyrococcus sp.,https://patents.google.com/patent/EP2159287A2/...,https://patents.google.com/patent/US7026526B2/en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,EP1337242B1,EP_1337242_B1_20110330,178-381-548-392-714,2011-03-30,{'simple_family': {'members': [{'document_id':...,"['C12N9/64', 'C12Q1/04', 'C12Q1/37', 'G01N33/5...",[{'text': 'The present inventors have develope...,[{'claims': [{'claim_text': ['A composition co...,"{'granted': True, 'grant_date': '2011-03-30', ...",GRANTED_PATENT,096-880-211-513-211,Physeter macrocephalus,https://patents.google.com/patent/JP2004518693...,https://patents.google.com/patent/EP1337242B1/en
2685,US8227231B2,US_8227231_B2_20120724,184-753-518-740-126,2012-07-24,{'simple_family': {'members': [{'document_id':...,"['C12N15/63', 'C07H21/02', 'C07H21/04', 'C07K1...",[{'text': 'Specified restriction endonucleases...,[{'claims': [{'claim_text': ['1. An isolated p...,"{'granted': True, 'grant_date': '2012-07-24', ...",GRANTED_PATENT,078-117-691-594-089,Anabaena sp. al09,https://patents.google.com/patent/US8685689B2/...,https://patents.google.com/patent/US8227231B2/en
2686,EP2761009B1,EP_2761009_B1_20150909,187-119-198-947-822,2015-09-09,{'simple_family': {'members': [{'document_id':...,"['C07H21/04', 'A01H1/00', 'C07K14/37', 'C07K14...",NaN,"[{'claims': [{'claim_text': [""A composition fo...","{'granted': True, 'grant_date': '2015-09-09', ...",GRANTED_PATENT,178-911-351-935-717,Odobenus rosmarus divergens,https://patents.google.com/patent/US9315825B2/...,https://patents.google.com/patent/EP2761009B1/en
2687,US8962556B2,US_8962556_B2_20150224,193-615-366-616-054,2015-02-24,{'s

In [6]:
#get from keys
#df_ids = list(df_keys['patent_num'])


In [12]:
# take id from URL (better than from the name)
df_ids = []
for ids in df_keys['patent_url_new']:
    patent_nr = ids.split("patent/",1)[1].split('/en',1)[0]
    df_ids.append(patent_nr)

In [16]:
# Scrape #
# this is taken and edited from https://github.com/ryanlstevens/google_patent_scraper
from urllib.request import Request, urlopen
import urllib.parse
from urllib.error import HTTPError 
from bs4 import BeautifulSoup
# json # 
import json
# errors #
#from .errors import * --> I don't know how to import these errors properly. 

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
#           Create scraper class
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
class scraper_class_edit:
    """
    Google scraper class used to scrape data from 'https://patents.google.com/'
    There are two primary ways to use the class:
        (1) Add list of patents to class and scrape all patents at once
        scraper=scraper_class() #<- Initialize class
        # ~ Add patents to list ~ #
        scraper.add_patents('US2668287A')
        scraper.add_patents('US266827A')
        # ~ Scrape all patents ~ #
        scraper.scrape_all_patents()
        # ~ Get results of scrape ~ #
        patent_1_parsed = scraper.parsed_patents['2668287A']
        patent_2_parsed = scraper.parsed_patents['266827A']
        (2) Scrape each patent individually
        scraper=scraper_class() #<- Initialize class
        # ~~ Scrape patents individually ~~ #
        patent_1 = 'US2668287A'
        patent_2 = 'US266827A'
        err_1, soup_1, url_1 = scraper.request_single_patent(patent_1)
        err_2, soup_2, url_2 = scraper.request_single_patent(patent_2)
        # ~ Parse results of scrape ~ #
        patent_1_parsed = scraper.get_scraped_data(soup_1,patent_1,url_1)
        patent_2_parsed = scraper.get_scraped_data(soup_2,patetn_2,url_2)
    Attributes:
        - list_of_patents (list) : patents to be scraped
        - scrape_status   (dict) : status of request using patent
        - parsed_patents  (dict) : result of parsing patent html
        - return_abstract (bool) : boolean for whether the code should return the abstract  
    """
    def __init__(self,return_abstract=False,return_fulltext=False,return_claims=False):
        self.list_of_patents = []
        self.scrape_status = {}
        self.parsed_patents = {}
        self.return_abstract = return_abstract
        self.return_fulltext = return_fulltext
        self.return_claims = return_claims

    def add_patents(self, patent):
        """Append patent to patent list attribute self.list_of_patents
        Inputs:
            - patent (str) : patent number 
        """
        # ~ Check if patent is string ~ #
        if not isinstance(patent,str):
            raise(PatentClassError("'patent' variable must be a string"))
        # ~ Append patent to list to be scrapped ~ #
        else:
            self.list_of_patents.append(patent)

    def delete_patents(self,patent):
        """Remove patent from patent list attribute self.list_of_patents
        Inputs:
            - patent (str) : patent number
        """
        # ~ Check if patent is in list ~ #
        if patent in self.list_of_patents:
            self.list_of_patents.pop(self.list_of_patents.index(patent))
        else:
            print('Patent {0} not in patent list'.format(patent))            


    def add_scrape_status(self,patent,success_value):
        """Add status of scrape to dictionary self.scrape_status"""
        self.scrape_status[patent] = success_value

    def request_single_patent(self,patent,url=False):
        """Calls request function to retreive google patent data and parses returned html using BeautifulSoup
        Returns: 
            - Status of scrape   <- String
            - Html of patent     <- BS4 object
        Inputs:
            - patent (str)  : if    url == False then patent is patent number
                              elif  url == True  then patent is google patent url
            - url    (bool) : determines whether patent is treated as patent number 
                                or google patent url
        """
        try:
            if not url:
                url='https://patents.google.com/patent/{0}/en'.format(patent)
            else:
                url=patent
            print(url)
            req = Request(url,headers={'User-Agent': 'Mozilla/5.0'})
            webpage = urlopen(req).read()
            soup = BeautifulSoup(webpage, features="lxml")
            return(('Success',soup,url))
        except HTTPError as e:
            print('Patent: {0}, Error Status Code : {1}'.format(patent,e.code))
            return(e.code,'',url)

    def parse_citation(self,single_citation):
        """Parses patent citation, returning results as a dictionary
        Returns (variables returned in dictionary, following are key names):  
            - patent_number (str)  : patent number
            - priority_date (str)  : priority date of patent
            - pub_date      (str)  : publication date of patent
        Inputs:
            - single_citation (str) : html string from citation section in google patent html
        """

        try:
            patent_number = single_citation.find('span',itemprop='publicationNumber').get_text()
        except:
            patent_number = ''
        # ~ Get priority date ~ #
        try:
            priority_date = single_citation.find('td',itemprop='priorityDate').get_text()
        except:
            priority_date = ''
        # ~ Get publication date ~ # 
        try:
            pub_date = single_citation.find('td',itemprop='publicationDate').get_text()
        except:
            pub_date
        return({'patent_number':patent_number,
                'priority_date':priority_date,
                'pub_date':pub_date})
    
    def tag_visible(element):
        """ This is somethign I might need to clean the full-text """
        if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
            return False
        if isinstance(element, Comment):
            return False
        return True
    
    
    def process_patent_html(self,soup):
        """ Parse patent html using BeautifulSoup module
        Returns (variables returned in dictionary, following are key names): 
            - application_number        (str)   : application number
            - inventor_name             (json)  : inventors of patent 
            - assignee_name_orig        (json)  : original assignees to patent
            - assignee_name_current     (json)  : current assignees to patent
            - pub_date                  (str)   : publication date
            - filing_date               (str)   : filing date
            - priority_date             (str)   : priority date
            - grant_date                (str)   : grant date
            - forward_cites_no_family   (json)  : forward citations that are not family-to-family cites
            - forward_cites_yes_family  (json)  : forward citations that are family-to-family cites
            - backward_cites_no_family  (json)  : backward citations that are not family-to-family cites
            - backward_cites_yes_family (json)  : backward citations that are family-to-family cites
        Inputs:
            - soup (str) : html string from of google patent html
            
        """
        try:
            inventor_name = [{'inventor_name':x.get_text()} for x in soup.find_all('dd',itemprop='inventor')]
        except:
            inventor_name = []
        # Assignee #
        try:
            assignee_name_orig = [{'assignee_name':x.get_text()} for x in soup.find_all('dd',itemprop='assigneeOriginal')]
        except:
            assignee_name_orig = []
        try:
            assignee_name_current = [{'assignee_name':x.get_text()} for x in soup.find_all('dd',itemprop='assigneeCurrent')]
        except:
            assignee_name_current = []
          
        # Publication Date #
        try:
            pub_date = soup.find('dd',itemprop='publicationDate').get_text()
        except:
            pub_date = ''
        # Application Number #
        try: 
            application_number = soup.find('dd',itemprop="applicationNumber").get_text()
        except:
            application_number = ''
        # Filing Date # 
        try:
            filing_date = soup.find('dd',itemprop='filingDate').get_text()
        except:
            filing_date = ''
        # Loop through all events #
        list_of_application_events = soup.find_all('dd',itemprop='events')
        priority_date = ''
        grant_date = ''
        for app_event in list_of_application_events:
            # Get information #
            try:
                title_info = app_event.find('span',itemprop='type').get_text()
                timeevent = app_event.find('time',itemprop='date').get_text()
                if title_info == 'priority':
                    priority_date = timeevent
                if title_info == 'granted':
                    grant_date = timeevent
                if title_info == 'publication' and pub_date=='':
                    pub_date = timeevent
            except:
                continue 

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
        #             Citations
        #
        # All citations are of the same format
        #   -Find all citations
        #   -If there are any citations, parse each individually using "parse_citation"
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
        # ~~~ Forward Citations (No Family to Family) ~~~ #
        found_forward_cites_orig=soup.find_all('tr', itemprop="forwardReferencesOrig")
        forward_cites_no_family=[]
        if len(found_forward_cites_orig)>0:
            for citation in found_forward_cites_orig:
                forward_cites_no_family.append(self.parse_citation(citation))

        # ~~~ Forward Citations (Yes Family to Family) ~~~ #
        found_forward_cites_family=soup.find_all('tr', itemprop="forwardReferencesFamily")
        forward_cites_yes_family=[]
        if len(found_forward_cites_family)>0:
            for citation in found_forward_cites_family:
                forward_cites_yes_family.append(self.parse_citation(citation))

        # ~~~ Backward Citations (No Family to Family) ~~~ #
        found_backward_cites_orig = soup.find_all('tr', itemprop='backwardReferences')
        backward_cites_no_family=[]
        if len(found_backward_cites_orig)>0:
            for citation in found_backward_cites_orig:
                backward_cites_no_family.append(self.parse_citation(citation))

        # ~~~ Backward Citations (Yes Family to Family) ~~~ #
        found_backward_cites_family = soup.find_all('tr', itemprop='backwardReferencesFamily')
        backward_cites_yes_family=[]
        if len(found_backward_cites_family)>0:
            for citation in found_backward_cites_family:
                backward_cites_yes_family.append(self.parse_citation(citation))

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
        #  Get abstract 
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
        abstract_text=''
        if self.return_abstract:
            # Get abstract # 
            abstract = soup.find('meta',attrs={'name':'DC.description'})
            # Get text 
            if abstract:
                abstract_text=abstract['content']
        
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
        #  Get full-text
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
        
        full_text = ''
        if self.return_fulltext:
            # Get full text #
            full_text = "".join([p.text for p in soup.find_all('ul', class_='description')])
            
            # Alternative
            if full_text == '': 
                full_text = "".join([p.text for p in soup.find_all('div', class_='description')])
               # full_text=fulltext['..'] #lets see
            
                
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
        #  Get claims
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
        
        claims = ''
        if self.return_claims:
            # Get claims #
            claims = "".join([p.text for p in soup.find_all('div', class_='claims')])
            
            # Alternative
            if claims == '': 
                claims = "".join([p.text for p in soup.find_all('ol', class_='claims')])
               # full_text=fulltext['..'] #lets see
        
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
        #  Return data as a dictionary
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
        return({'inventor_name':json.dumps(inventor_name),
                'assignee_name_orig':json.dumps(assignee_name_orig),
                'assignee_name_current':json.dumps(assignee_name_current),
                'pub_date':pub_date,
                'priority_date':priority_date,
                'grant_date':grant_date,
                'filing_date':filing_date,
                'forward_cite_no_family':json.dumps(forward_cites_no_family),
                'forward_cite_yes_family':json.dumps(forward_cites_yes_family),
                'backward_cite_no_family':json.dumps(backward_cites_no_family),
                'backward_cite_yes_family':json.dumps(backward_cites_yes_family),
                'abstract_text':abstract_text,
                'full_text':full_text,
                'claims':claims})

    def get_scraped_data(self,soup,patent,url):
        # ~~ Parse individual patent ~~ #
        parsing_individ_patent = self.process_patent_html(soup)
        # ~~ Add url + patent to dictionary ~~ #
        parsing_individ_patent['url'] = url
        parsing_individ_patent['patent'] = patent
        # ~~ Return patent info ~~ #
        return(parsing_individ_patent)


    def scrape_all_patents(self):
        """ Scrapes all patents in list self.list_of_patents using function "request_single_patent". 
        If you want to scrape a single patent without adding it to the class variable, 
            use "request_single_patent" function as a method on the class. See the doc string
            in the class module for an example.
        """
        # ~ Check if there are any patents ~ #
        if len(self.list_of_patents)==0:
            raise(NoPatentsError("no patents to scrape specified in 'patent' variable: add patent using class.add_patents([<PATENTNUMBER>])"))
        # ~ Loop through list of patents and scrape them ~ #
        else:
            n=1
            for patent in self.list_of_patents:
                
                print(n)
                sleep(randint(2,4))
                
                error_status, soup, url = self.request_single_patent(patent)
                # ~ Add scrape status variable ~ #
                
                self.add_scrape_status(patent,error_status)
                
                if error_status=='Success':
                    self.parsed_patents[patent] = self.get_scraped_data(soup,patent,url)
                    n = n+1
                else:
                    self.parsed_patents[patent] = {}
                    n = n+1

## Test scrape

In [18]:
# Scrape one patent
scraper=scraper_class_edit(return_fulltext=True)  #<- TURN ON ABSTRACT TEXT or fulltext

# ~~ Scrape patents individually ~~ #
patent_1 = 'EP2199396A1'
err_1, soup_1, url_1 = scraper.request_single_patent(patent_1)

# ~ Parse results of scrape ~ #
patent_1_parsed = scraper.get_scraped_data(soup_1,patent_1,url_1)

# ~ Print abstract text ~ #
print(patent_1_parsed['full_text'])

https://patents.google.com/patent/EP2199396A1

   The present invention relates generally to the field of molecular biology and concerns a method for improving various plant characteristics by modulating expression in a plant of a nucleic acid encoding a GRP (Growth Related Protein). The present invention also concerns plants having modulated expression of a nucleic acid encoding a GRP polypeptide, which plants have improved characteristics relative to corresponding wild type plants or other control plants. The invention also provides constructs useful in the methods of the invention.
    The ever-increasing world population and the dwindling supply of arable land available for agriculture fuels research towards increasing the efficiency of agriculture. Conventional means for crop and horticultural improvements utilise selective breeding techniques to identify plants having desirable characteristics. However, such selective breeding techniques have several drawbacks, namely that these 

In [19]:
# ~ Print inventors of patent US2668287A ~ #
for inventor in json.loads(patent_1_parsed['assignee_name_orig']):
      print(inventor)

{'assignee_name': 'CropDesign NV'}


## Scraping

In [17]:
# Scrape the list of patents
# ~ Initialize edited scraper class ~ #
scraper=scraper_class_edit(return_fulltext=True,return_claims=True)

# ~ Add patents to list ~ #
for id_ in df_ids: 
    scraper.add_patents(id_)


#scraper.add_patents('US2668287A')
#scraper.add_patents('US266827A')

# ~ Scrape all patents ~ #
scraper.scrape_all_patents()

# ~ Get results of scrape ~ #
#patent_1_parsed = scraper.parsed_patents['US2668287A']
#patent_2_parsed = scraper.parsed_patents['US266827A']

# ~ Print inventors of patent US2668287A ~ #
#for inventor in json.loads(patent_1_parsed['inventor_name']):
  #print('Patent inventor : {0}'.format(inventor['inventor_name']))

    
    
# looking at the data in the return dictionary

#for k,v in scraper.parsed_patents.items():
    #
    

1
https://patents.google.com/patent/TWI352737B/en
2
https://patents.google.com/patent/US9410134B2/en
3
https://patents.google.com/patent/US9422558B2/en
4
https://patents.google.com/patent/US8420312B2/en
5
https://patents.google.com/patent/US7026526B2/en
6
https://patents.google.com/patent/US8067178B2/en
7
https://patents.google.com/patent/US8859237B2/en
8
https://patents.google.com/patent/JP2006197947A/en
9
https://patents.google.com/patent/US9096889B2/en
10
https://patents.google.com/patent/US9670521B2/en
11
https://patents.google.com/patent/CA3075765A1/en
12
https://patents.google.com/patent/US7863437B2/en
13
https://patents.google.com/patent/KR101316084B1/en
14
https://patents.google.com/patent/EP1259608B1/en
15
https://patents.google.com/patent/EP2331695B1/en
16
https://patents.google.com/patent/AU2006319259A1/en
17
https://patents.google.com/patent/CA2574727A1/en
18
https://patents.google.com/patent/WO2000023615A2/en
19
https://patents.google.com/patent/US9765126B2/en
20
https://p

https://patents.google.com/patent/US5919653A/en
156
https://patents.google.com/patent/US9493842B2/en
157
https://patents.google.com/patent/ES2467155T3/en
158
https://patents.google.com/patent/US6593135B2/en
159
https://patents.google.com/patent/US9370606B2/en
160
https://patents.google.com/patent/JP3507890B2/en
161
https://patents.google.com/patent/EP1326890B1/en
162
https://patents.google.com/patent/US7932058B2/en
163
https://patents.google.com/patent/US7923233B1/en
164
https://patents.google.com/patent/ZA201205427B/en
165
https://patents.google.com/patent/JP4280390B2/en
166
https://patents.google.com/patent/CN105612426B/en
167
https://patents.google.com/patent/US10463748B2/en
168
https://patents.google.com/patent/KR101227432B1/en
169
https://patents.google.com/patent/US8298769B2/en
170
https://patents.google.com/patent/EP3782465B1/en
171
https://patents.google.com/patent/KR101859685B1/en
172
https://patents.google.com/patent/US9441020B2/en
173
https://patents.google.com/patent/CA2439

308
https://patents.google.com/patent/WO2009059056A2/en
309
https://patents.google.com/patent/US8975061B2/en
310
https://patents.google.com/patent/US8969301B2/en
311
https://patents.google.com/patent/ZA200803338B/en
312
https://patents.google.com/patent/US11059865B2/en
313
https://patents.google.com/patent/US7666852B2/en
314
https://patents.google.com/patent/EP2157170B1/en
315
https://patents.google.com/patent/CA2473082A1/en
316
https://patents.google.com/patent/US10703786B2/en
317
https://patents.google.com/patent/AU2005322640B2/en
318
https://patents.google.com/patent/EP3262182B1/en
319
https://patents.google.com/patent/CA3150733A1/en
320
https://patents.google.com/patent/JP2021112164A/en
321
https://patents.google.com/patent/AU2006304100B2/en
322
https://patents.google.com/patent/ZA200302739B/en
323
https://patents.google.com/patent/US7683036B2/en
324
https://patents.google.com/patent/WO2019217327A1/en
325
https://patents.google.com/patent/RU2580047C2/en
326
https://patents.google.c

460
https://patents.google.com/patent/US9181566B2/en
461
https://patents.google.com/patent/US8975474B2/en
462
https://patents.google.com/patent/CA2611513A1/en
463
https://patents.google.com/patent/JP4953487B1/en
464
https://patents.google.com/patent/WO2010091692A2/en
465
https://patents.google.com/patent/US8691771B2/en
466
https://patents.google.com/patent/ZA201007180B/en
467
https://patents.google.com/patent/JP4452834B2/en
468
https://patents.google.com/patent/US8188255B2/en
469
https://patents.google.com/patent/US9963686B2/en
470
https://patents.google.com/patent/EP2463377B1/en
471
https://patents.google.com/patent/US8093043B2/en
472
https://patents.google.com/patent/US9206444B2/en
473
https://patents.google.com/patent/ES2291071B1/en
474
https://patents.google.com/patent/CN101809146B/en
475
https://patents.google.com/patent/US7056740B2/en
476
https://patents.google.com/patent/US9850298B2/en
477
https://patents.google.com/patent/US5866755A/en
478
https://patents.google.com/patent/ZA20

613
https://patents.google.com/patent/JP5931584B2/en
614
https://patents.google.com/patent/AU2004290076B2/en
615
https://patents.google.com/patent/AU2010329917B2/en
616
https://patents.google.com/patent/US9909123B2/en
617
https://patents.google.com/patent/US8431692B2/en
618
https://patents.google.com/patent/IT1175508B/en
619
https://patents.google.com/patent/AU2010203031B2/en
620
https://patents.google.com/patent/EP3077517B1/en
621
https://patents.google.com/patent/US10913939B2/en
622
https://patents.google.com/patent/TWI638047B/en
623
https://patents.google.com/patent/US9283287B2/en
624
https://patents.google.com/patent/US6727226B2/en
625
https://patents.google.com/patent/EP2439271B1/en
626
https://patents.google.com/patent/WO2001096388A2/en
627
https://patents.google.com/patent/MX338065B/en
628
https://patents.google.com/patent/EP2798064B1/en
629
https://patents.google.com/patent/JP2000350584A/en
630
https://patents.google.com/patent/JP6773268B2/en
631
https://patents.google.com/pate

...

1967
https://patents.google.com/patent/ZA200903022B/en
1968
https://patents.google.com/patent/US7297492B2/en
1969
https://patents.google.com/patent/CA2555509A1/en
1970
https://patents.google.com/patent/WO2017048850A1/en
1971
https://patents.google.com/patent/JP5354710B2/en
1972
https://patents.google.com/patent/AU2013349725B2/en
1973
https://patents.google.com/patent/US7226993B2/en
1974
https://patents.google.com/patent/JP5354559B2/en
1975
https://patents.google.com/patent/US20050123940A1/en
1976
https://patents.google.com/patent/AU2002315592B2/en
1977
https://patents.google.com/patent/EP2842578B1/en
1978
https://patents.google.com/patent/CA2750007A1/en
1979
https://patents.google.com/patent/JP2008182983A/en
1980
https://patents.google.com/patent/WO2013114363A2/en
1981
https://patents.google.com/patent/US7314974B2/en
1982
https://patents.google.com/patent/CN101675167B/en
1983
https://patents.google.com/patent/JP2006180886A/en
1984
https://patents.google.com/patent/JP6033632B2/en
1985
h

2117
https://patents.google.com/patent/US8440391B2/en
2118
https://patents.google.com/patent/US6471964B1/en
2119
https://patents.google.com/patent/JP2005198600A/en
2120
https://patents.google.com/patent/AU2005327520B2/en
2121
https://patents.google.com/patent/JP5177996B2/en
2122
https://patents.google.com/patent/US8343743B2/en
2123
https://patents.google.com/patent/US9464313B2/en
2124
https://patents.google.com/patent/WO2012024678A2/en
2125
https://patents.google.com/patent/JP2020054250A/en
2126
https://patents.google.com/patent/RU2501564C2/en
2127
https://patents.google.com/patent/EP3786290A1/en
2128
https://patents.google.com/patent/WO2009109631A1/en
2129
https://patents.google.com/patent/ZA201403938B/en
2130
https://patents.google.com/patent/US7807392B1/en
2131
https://patents.google.com/patent/US10563211B2/en
2132
https://patents.google.com/patent/US7893196B2/en
2133
https://patents.google.com/patent/JP6670831B2/en
2134
https://patents.google.com/patent/US8618070B2/en
2135
https://

2267
https://patents.google.com/patent/US8158770B2/en
2268
https://patents.google.com/patent/EP2376638B1/en
2269
https://patents.google.com/patent/JP4693998B2/en
2270
https://patents.google.com/patent/US10342798B2/en
2271
https://patents.google.com/patent/US11118180B2/en
2272
https://patents.google.com/patent/US5643781A/en
2273
https://patents.google.com/patent/US9567581B2/en
2274
https://patents.google.com/patent/ZA200501454B/en
2275
https://patents.google.com/patent/MA32104B1/en
2276
https://patents.google.com/patent/US8618354B2/en
2277
https://patents.google.com/patent/CN106132423B/en
2278
https://patents.google.com/patent/US10750711B2/en
2279
https://patents.google.com/patent/US20030129723A1/en
2280
https://patents.google.com/patent/JP5684984B2/en
2281
https://patents.google.com/patent/JP6267328B2/en
2282
https://patents.google.com/patent/CN100340668C/en
2283
https://patents.google.com/patent/US8445242B2/en
2284
https://patents.google.com/patent/JP7360154B2/en
2285
https://patents.

2417
https://patents.google.com/patent/US10385103B2/en
2418
https://patents.google.com/patent/EP1573000B1/en
2419
https://patents.google.com/patent/US7771995B2/en
2420
https://patents.google.com/patent/US11028148B2/en
2421
https://patents.google.com/patent/US9518277B2/en
2422
https://patents.google.com/patent/JP2010263812A/en
2423
https://patents.google.com/patent/GB2464888B/en
2424
https://patents.google.com/patent/EP2133364A1/en
2425
https://patents.google.com/patent/EA028648B1/en
2426
https://patents.google.com/patent/EP2995626B1/en
2427
https://patents.google.com/patent/US10570407B2/en
2428
https://patents.google.com/patent/US7790155B2/en
2429
https://patents.google.com/patent/FR2908136A1/en
2430
https://patents.google.com/patent/GB0414987D0/en
2431
https://patents.google.com/patent/JP2019151579A/en
2432
https://patents.google.com/patent/US8546078B2/en
2433
https://patents.google.com/patent/AU2014236683B2/en
2434
https://patents.google.com/patent/EP3383361B1/en
2435
https://patents

2567
https://patents.google.com/patent/US9066967B2/en
2568
https://patents.google.com/patent/ZA201202248B/en
2569
https://patents.google.com/patent/US11085057B2/en
2570
https://patents.google.com/patent/JP2021101628A/en
2571
https://patents.google.com/patent/US6783958B2/en
2572
https://patents.google.com/patent/US10189880B2/en
2573
https://patents.google.com/patent/US7847156B2/en
2574
https://patents.google.com/patent/US6624288B1/en
2575
https://patents.google.com/patent/US10906990B2/en
2576
https://patents.google.com/patent/US8236327B2/en
2577
https://patents.google.com/patent/US8865467B2/en
2578
https://patents.google.com/patent/WO2014105840A1/en
2579
https://patents.google.com/patent/EA015591B1/en
2580
https://patents.google.com/patent/JP6489017B2/en
2581
https://patents.google.com/patent/US9353359B2/en
2582
https://patents.google.com/patent/US8211643B2/en
2583
https://patents.google.com/patent/US8206978B2/en
2584
https://patents.google.com/patent/WO2005111244A2/en
2585
https://pate

In [18]:
# checking if it all worked
ft = 0
c = 0

for k,v in scraper.parsed_patents.items():
    for key, value in v.items():
        if key == 'claims':
            if value == '': 
                print(k + '  no claims')
            else: 
                #print('ok')
                c = c+1
        if key == 'full_text':
            if value == '':
                print(k + '  no full_text')
            else: 
                #print('ok')
                ft = ft+1

TWI352737B  no full_text
TWI352737B  no claims
JP2006197947A  no full_text
JP2006197947A  no claims
KR101316084B1  no full_text
KR101316084B1  no claims
JP5428527B2  no full_text
JP5428527B2  no claims
ZA201204066B  no full_text
ZA201204066B  no claims
GB0915403D0  no full_text
GB0915403D0  no claims
JP2021122189A  no full_text
JP2021122189A  no claims
JP2003245074A  no full_text
JP2003245074A  no claims
JP6541173B2  no full_text
JP6541173B2  no claims
JP6206893B2  no full_text
JP6206893B2  no claims
ZA200704389B  no full_text
ZA200704389B  no claims
JP7203744B2  no full_text
JP7203744B2  no claims
NI201500132A  no full_text
NI201500132A  no claims
GB2467722B  no full_text
GB2467722B  no claims
JP2022029888A  no full_text
JP2022029888A  no claims
PH12014502382B1  no full_text
PH12014502382B1  no claims
JP2007166921A  no full_text
JP2007166921A  no claims
KR101047192B1  no full_text
KR101047192B1  no claims
JP2010006745A  no full_text
JP2010006745A  no claims
GB0025804D0  no full_text
G

In [22]:
ft

1959

In [23]:
c

1964

In [20]:
# Saving data as json
with open(R'X:\5_Research\Paul\data\genes_of_technology\patent_fulltexts.json', 'w') as json_file:
    json.dump(scraper.parsed_patents, json_file)

## Add the fulltexts to the df

In [24]:
# Open data for inspection
with open(R'X:\5_Research\Paul\data\genes_of_technology\patent_fulltexts.json') as json_file:
    a = json.load(json_file)

In [25]:
patent_nr = []
fulltext = []

for k, v in a.items():
    patent_nr.append(k)
    
    if 'full_text' in v.keys(): 
        fulltext.append(v['full_text'])
        #print(len(v['full_text']))
    else: 
        fulltext.append('')

In [30]:
len(patent_nr)

2689

In [31]:
df_keys

,patent_id,doc_key,lens_id_new,date_published,families,codes,abstract,claims,legal_status,publication_type,lens_id_old,organism,patent_url_old,patent_url_new
0,TWI352737B,TW_I352737_B_20111121,000-950-825-379-991,2011-11-21,{'simple_family': {'members': [{'document_id':...,"['C07H21/04', 'A01K67/027', 'C12N9/16', 'C12N9...",[{'text': 'The current invention reports a pro...,NaN,"{'granted': True, 'grant_date': '2011-11-21', ...",GRANTED_PATENT,154-387-245-471-34X,Aequorea victoria,https://patents.google.com/patent/JP2010531647...,https://patents.google.com/patent/TWI352737B/en
1,US9410134B2,US_9410134_B2_20160809,000-959-609-762-989,2016-08-09,{'simple_family': {'members': [{'document_id':...,"['C12N15/113', 'A01H3/04', 'A01N57/16', 'A01N5...",[{'text': 'The present invention relates to a ...,[{'claims': [{'claim_text': ['1. A nucleic aci...,"{'granted': True, 'grant_date': '2016-08-09', ...",GRANTED_PATENT,000-959-609-762-989,Odobenus rosmarus divergens,https://patents.google.com/patent/US9410134B2/...,https://patents.google.com/patent/US9410134B2/...
2,US9422558B2,US_9422558_B2_20160823,003-170-265-500-408,2016-08-23,{'simple_family': {'members': [{'document_id':...,"['C07H21/04', 'C12Q1/68']",[{'text': 'The present invention provides nove...,"[{'claims': [{'claim_text': [""1. A method of p...","{'granted': True, 'grant_date': '2016-08-23', ...",GRANTED_PATENT,003-170-265-500-408,Rhizophora mucronata,https://patents.google.com/patent/US9422558B2/...,https://patents.google.com/patent/US9422558B2/...
3,US8420312B2,US_8420312_B2_20130416,004-053-513-540-817,2013-04-16,{'simple_family': {'members': [{'document_id':...,"['C12N15/29', 'C12N5/04', 'C12N15/82', 'C12N15...",[{'text': 'Described herein is a combination c...,[{'claims': [{'claim_text': ['1. A method to d...,"{'granted': True, 'grant_date': '2013-04-16', ...",GRANTED_PATENT,004-053-513-540-817,Trichechus manatus latirostris,https://patents.google.com/patent/US8420312B2/...,https://patents.google.com/patent/US8420312B2/...
4,US7026526B2,US_7026526_B2_20060411,004-902-052-988-996,2006-04-11,{'simple_family': {'members': [{'document_id':...,"['C12Q1/68', 'G01N33/53']","[{'text': ""Methods and constructs for the intr...",[{'claims': [{'claim_text': ['1. A DNA constru...,"{'granted': True, 'grant_date': '2006-04-11', ...",GRANTED_PATENT,045-496-187-872-188,Pyrococcus sp.,https://patents.google.com/patent/EP2159287A2/...,https://patents.google.com/patent/US7026526B2/en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,EP1337242B1,EP_1337242_B1_20110330,178-381-548-392-714,2011-03-30,{'simple_family': {'members': [{'document_id':...,"['C12N9/64', 'C12Q1/04', 'C12Q1/37', 'G01N33/5...",[{'text': 'The present inventors have develope...,[{'claims': [{'claim_text': ['A composition co...,"{'granted': True, 'grant_date': '2011-03-30', ...",GRANTED_PATENT,096-880-211-513-211,Physeter macrocephalus,https://patents.google.com/patent/JP2004518693...,https://patents.google.com/patent/EP1337242B1/en
2685,US8227231B2,US_8227231_B2_20120724,184-753-518-740-126,2012-07-24,{'simple_family': {'members': [{'document_id':...,"['C12N15/63', 'C07H21/02', 'C07H21/04', 'C07K1...",[{'text': 'Specified restriction endonucleases...,[{'claims': [{'claim_text': ['1. An isolated p...,"{'granted': True, 'grant_date': '2012-07-24', ...",GRANTED_PATENT,078-117-691-594-089,Anabaena sp. al09,https://patents.google.com/patent/US8685689B2/...,https://patents.google.com/patent/US8227231B2/en
2686,EP2761009B1,EP_2761009_B1_20150909,187-119-198-947-822,2015-09-09,{'simple_family': {'members': [{'document_id':...,"['C07H21/04', 'A01H1/00', 'C07K14/37', 'C07K14...",NaN,"[{'claims': [{'claim_text': [""A composition fo...","{'granted': True, 'grant_date': '2015-09-09', ...",GRANTED_PATENT,178-911-351-935-717,Odobenus rosmarus divergens,https://patents.google.com/patent/US9315825B2/...,https://patents.google.com/patent/EP2761009B1/en
2687,US8962556B2,US_8962556_B2_20150224,193-615-366-616-054,2015-02-24,{'s

In [33]:
df_keys_fulltxt = df_keys.merge(pd.DataFrame(zip(patent_nr,fulltext), columns = ['patent_id', 'fulltext']), how = 'inner')

In [34]:
df_keys_fulltxt

,patent_id,doc_key,lens_id_new,date_published,families,codes,abstract,claims,legal_status,publication_type,lens_id_old,organism,patent_url_old,patent_url_new,fulltext
0,TWI352737B,TW_I352737_B_20111121,000-950-825-379-991,2011-11-21,{'simple_family': {'members': [{'document_id':...,"['C07H21/04', 'A01K67/027', 'C12N9/16', 'C12N9...",[{'text': 'The current invention reports a pro...,NaN,"{'granted': True, 'grant_date': '2011-11-21', ...",GRANTED_PATENT,154-387-245-471-34X,Aequorea victoria,https://patents.google.com/patent/JP2010531647...,https://patents.google.com/patent/TWI352737B/en,
1,US9410134B2,US_9410134_B2_20160809,000-959-609-762-989,2016-08-09,{'simple_family': {'members': [{'document_id':...,"['C12N15/113', 'A01H3/04', 'A01N57/16', 'A01N5...",[{'text': 'The present invention relates to a ...,[{'claims': [{'claim_text': ['1. A nucleic aci...,"{'granted': True, 'grant_date': '2016-08-09', ...",GRANTED_PATENT,000-959-609-762-989,Odobenus rosmarus divergens,https://patents.google.com/patent/US9410134B2/...,https://patents.google.com/patent/US9410134B2/...,\nThe text file entitled “Vossius_03USnew_ST25...
2,US9422558B2,US_9422558_B2_20160823,003-170-265-500-408,2016-08-23,{'simple_family': {'members': [{'document_id':...,"['C07H21/04', 'C12Q1/68']",[{'text': 'The present invention provides nove...,"[{'claims': [{'claim_text': [""1. A method of p...","{'granted': True, 'grant_date': '2016-08-23', ...",GRANTED_PATENT,003-170-265-500-408,Rhizophora mucronata,https://patents.google.com/patent/US9422558B2/...,https://patents.google.com/patent/US9422558B2/...,\nThis application claims benefit under 35 USC...
3,US8420312B2,US_8420312_B2_20130416,004-053-513-540-817,2013-04-16,{'simple_family': {'members': [{'document_id':...,"['C12N15/29', 'C12N5/04', 'C12N15/82', 'C12N15...",[{'text': 'Described herein is a combination c...,[{'claims': [{'claim_text': ['1. A method to d...,"{'granted': True, 'grant_date': '2013-04-16', ...",GRANTED_PATENT,004-053-513-540-817,Trichechus manatus latirostris,https://patents.google.com/patent/US8420312B2/...,https://patents.google.com/patent/US8420312B2/...,\nCROSS-REFERENCE TO RELATED APPLICATIONS\nThi...
4,US7026526B2,US_7026526_B2_20060411,004-902-052-988-996,2006-04-11,{'simple_family': {'members': [{'document_id':...,"['C12Q1/68', 'G01N33/53']","[{'text': ""Methods and constructs for the intr...",[{'claims': [{'claim_text': ['1. A DNA constru...,"{'granted': True, 'grant_date': '2006-04-11', ...",GRANTED_PATENT,045-496-187-872-188,Pyrococcus sp.,https://patents.google.com/patent/EP2159287A2/...,https://patents.google.com/patent/US7026526B2/en,\nThis application claims priority to U.S. Ser...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,EP1337242B1,EP_1337242_B1_20110330,178-381-548-392-714,2011-03-30,{'simple_family': {'members': [{'document_id':...,"['C12N9/64', 'C12Q1/04', 'C12Q1/37', 'G01N33/5...",[{'text': 'The present inventors have develope...,[{'claims': [{'claim_text': ['A composition co...,"{'granted': True, 'grant_date': '2011-03-30', ...",GRANTED_PATENT,096-880-211-513-211,Physeter macrocephalus,https://patents.google.com/patent/JP2004518693...,https://patents.google.com/patent/EP1337242B1/en,\n\nBackground Of The Invention\n\n\nField Of ...
2685,US8227231B2,US_8227231_B2_20120724,184-753-518-740-126,2012-07-24,{'simple_family': {'members': [{'document_id':...,"['C12N15/63', 'C07H21/02', 'C07H21/04', 'C07K1...",[{'text': 'Specified restriction endonucleases...,[{'claims': [{'claim_text': ['1. An isolated p...,"{'granted': True, 'grant_date': '2012-07-24', ...",GRANTED_PATENT,078-117-691-594-089,Anabaena sp. al09,https://patents.google.com/patent/US8685689B2/...,https://patents.google.com/patent/US8227231B2/en,\nCROSS REFERENCE\nThis application is a §371 ...
2686,EP2761009B1,EP_2761009_B1_20150909,187-119-198-947-822,2015-09-09,{'simple_family': {'members': [{'document_id':...,"['C07H21/04', 'A01H1/00', 'C07K14/37', 'C07K14...",NaN,"[{'claims': [{'claim_text'

In [35]:
df_keys_fulltxt.to_csv(R'X:\5_Research\Paul\data\genes_of_technology\full_patent_data_lens_marine_2689_familychecked_fulltxts.csv')